# FUNZIONI DASK_CUDF CHE NON VANNO (PER FARE CONFRONTO)

## Questa è la sezione da portare per far vedere i limiti di dask_cudf

### Conversione colonna da stringa a datetime

Errore che restituisce:
"TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy()."

In [ ]:
#CONVERSIONE COLONNE SU DASK_CUDF 
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}
cols = {
    0:'day',
    1:'month',
    2:'year'
}
a = base.split('gas_start_date', 2)
a = a.rename(columns=cols)
a['month'] = a['month'].str.lower().map(mesi)
dd.to_datetime(a).head()

### Conversione colonna da stringa in formato "1,22 unita_misura" a float

In [ ]:
#Ottenere unità di misura
misure = base.split('average_gas_bill_cost', 1)[1].value_counts().index.compute()[0]
#rimozione unità di misura
base.strip(['average_gas_bill_cost'], '€/smc')['average_gas_bill_cost']
#sostituzione della , con .
base.replace(['average_gas_bill_cost'], ',', '.', regex=True)['average_gas_bill_cost']

#### Le operazioni preliminari vanno a buon fine, qua si aprono più scenari (fallimentari)

questa soluzione è quella corretta su dask normale, utilizzando dask_cudf però genera un errore dovuto alla conversione implicita di un array in un numpy array, cosa non fattibile all'interno della GPU quindi consiglia di costruire esplicitamente una matrice tramite Cupy (Cuda Python) ma queste sono operazioni interne alla libreria

In [ ]:
dd.to_numeric(base.get_df()['average_gas_bill_cost'])
 

Qua da 2 diverse tipologie di errori: una dovuta ai metadati e l'altra alla presenza di valori non convertibili, anche togliedo tutti i valori nan con [~base['average_gas_bill_cost'].isna()] la situazione non cambia

In [ ]:
base.get_df()['average_gas_bill_cost'].map_partitions(cudf.to_numeric, 'raise')

# Pipeline

In [ ]:
''' 
+++++ UTILITY ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code for the customer that owns this utility
    customer_code --> String --> Combined with user_code provides a unique identifier for the utility. Even this field is anonymized
    city --> String --> City where the utility is located
    address --> String --> (Anonymized) address of the utility location
'''

In [ ]:
''' 
+++++ CUSTOMER ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code that identifies the customer
    nominative --> String --> (Anonymized) customer name
    sex --> String --> Sex of the customer. It could be ‘M’, ‘F’, ‘P’, with ‘P’ denoting that the customer is a commercial activity (VAT number)
    age --> Int --> Age of the customer, set to null for commercial activities (sex = ‘P’). Its value must be >= 18
'''

In [ ]:
''' 
+++++ INVOICE ATTRIBUTES +++++
    
    bill_id --> Int --> Invoice identifier
    F1_kWh --> Float --> kWh of electricity consumed in the F1 time slot
    F2_kWh --> Float --> kWh of electricity consumed in the F2 time slot
    F3_kWh --> Float --> kWh of electricity consumed in the F3 time slot 
    date --> Date --> Start date
    light_start_date --> Date --> Start date of electricity invoice
    light_end_date --> Date --> End date of electricity invoice
    tv --> Float --> Television fee to pay
    gas_amount --> Float --> Gas fee to pay
    gas_average_cost --> Float --> Average cost of gas
    light_average_cost --> Float --> Average cost of electricity
    emission_date --> Date --> Emission date
    supply_type --> String --> Supply type (‘light’, ‘gas’, ‘gas and light’)
    gas_start_date --> Date --> Start date of gas invoice
    gas_end_date --> Date --> End date of gas invoice
    extra_fees --> Float --> Extra fees to pay
    gas_consumption --> Float --> Consumed gas
    light_consumption --> Float --> Consumed electricity
    gas_offer --> Float --> Name of the subscribed gas plan (anonymized)
    light_offer_type --> String --> Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
    light_offer --> String --> Name of the subscribed electricity plan (anonymized)
    total_amount --> Float --> gas_amount + light_amount + extra_fees
    howmuch_pay --> Float --> Overall amount to pay, computed as total_amount + tv
    light_amount --> Float --> Amount to pay for the electricity
    average_unit_light_cost --> Float --> Average cost for electricity
    average_light_bill_cost --> Float --> Average cost for the electricity invoice
    average_unit_gas_cost --> Float --> Average cost for gas
    average_gas_bill_cost --> Float --> Average cost for the gas invoice
    billing_frequency --> String --> Billing frequency (‘monthly’, ‘quarterly’, etc.)
    bill_type --> String --> Kind of invoice (False means a “standard bill”)
    gas_system_charges --> Float --> Extra gas fees
    light_system_charges --> Float --> Extra electricity fees
    gas_material_cost --> Float --> Costs for gas
    light_transport_cost --> Float --> Extra electricity fees
    gas_transport_cost --> Float --> Extra gas fees
    light_material_cost --> Float --> Costs for electricity
'''

In [ ]:
''' 
+++++ DATA INGESTION +++++
    • Read data from its source
    • Study how to deal with data that doesn’t fit in memory (!) --> Da chiede a Gagliardelli
    • Locate missing values
    • Locate outliers
    • Sort data
'''

# Loading Dataset

In [1]:
import dask_cudf as dc
import dask.dataframe as dd
import dask.array as da
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import cudf
import numpy as np
#import graphviz
from base_dask_cudf import BaseDfBench
base = BaseDfBench("DASK")
# base = BaseDfBench()

In [2]:
# READ DATA FROM ITS SOURCE
# --- remove _sample one day

# We can use bill_id as the index for our dataset --> meglio di no
# base.load_dataset('/data/invoices_sample.csv', 'csv', index_col='bill_id')

# Uncomment the line under this one to load the pre-converted dataset
base.load_dataset('parquet', path='data/mod_parquet')
# base.load_dataset('parquet')
df = base.get_df()
columns = base.get_columns()
base.npartitions()

57

In [3]:
# Dataset's rows
# I commented the real computation because it takes a lot of time
# dataset_rows = base.get_df().shape[0].compute()
dataset_rows = 10497143
dataset_rows

In [ ]:
#verica di user_code+customer_code = PK utility
#PK customer = user_code
#PK invoice = bill_id
df.groupby(by=['user_code', 'customer_code'])['bill_id'].count().compute()

## Load big Dataset (DEPRECATED)

In [ ]:
# STUDY HOW TO DEAL WITH DATA THAT DON'T FIT IN MEMORY (!)
# Bella storia
!du -sh '/data/invoices_sample.csv'

In [ ]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

In [ ]:
cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("15GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("10GB"),
)
client = Client(cluster)
client

In [ ]:
%%time
ddf = dask_cudf.read_csv("/data/invoices.csv", chunksize="1 GiB")
print(f"DF partitions: {ddf.npartitions}")
print(f"DF: {ddf}")

# UTILS

In [48]:
def get_col_by_pattern(pat):
    ret = []
    for c in base.get_columns():
        if pat in c.split('_'):
            ret += [c]
    return ret

In [ ]:
# CONVERTING ALL THE STRING COLUMNS TO OBJECT COLUMN
# THIS BECAUSE STRING COLUMNS CAUSE A LOT OF ERRORS WHEN PROCESSED AS STRINGS
for k,v in base.get_columns_types().items():
    if v == 'string':
        base[k] = base[k].astype('object')
    elif v == 'Float64':
        base[k] = base[k].astype('float')

In [ ]:
# Seguendo il GroundTruth mystico
mappone = {'bill_id': 'int64',
 'F1_kWh': 'float64',
 'F2_kWh': 'float64',
 'F3_kWh': 'float64',
 'city': 'object',
 'address': 'object',
 'nominative': 'object',
 'sex': 'object',
 'age': 'int64',
 'user_code': 'object',
 'date': 'datetime64[ns]',
 'light_start_date': 'datetime64[ns]',
 'light_end_date': 'datetime64[ns]',
 'tv': 'float64',
 'gas_amount': 'float64',
 'gas_average_cost': 'float64',
 'light_average_cost': 'float64',
 'emission_date': 'datetime64[ns]',
 'supply_type': 'object',
 'gas_start_date': 'datetime64[ns]',
 'gas_end_date': 'datetime64[ns]',
 'extra_fees': 'float64',
 'gas_consumption': 'float64',
 'light_consumption': 'float64',
 'gas_offer': 'float64',
 'light_offer_type': 'object',
 'light_offer': 'object',
 'howmuch_pay': 'float64',
 'total_amount': 'float64',
 'light_amount': 'float64',
 'average_unit_light_cost': 'float64',
 'average_light_bill_cost': 'float64',
 'average_unit_gas_cost': 'float64',
 'average_gas_bill_cost': 'float64',
 'customer_code': 'object',
 'billing_frequency': 'object',
 'bill_type': 'object',
 'gas_system_charges': 'float64',
 'light_system_charges': 'float64',
 'gas_material_cost': 'float64',
 'light_transport_cost': 'float64',
 'gas_transport_cost': 'float64',
 'light_material_cost': 'float64'
}

In [ ]:
#Check sui tipi di dato attuali
type_dict = base.get_columns_types()
for k,v in type_dict.items():
    if v != mappone[k]:
        print(f'mismatch in {k} with values: {v}, {mappone[k]}')

In [ ]:
# Trovare le tipologie di colonne a seconda della tipologia
int_cols = base.col_type(find=['int'])
string_cols = base.col_type(find=['string']) #object
float_cols = base.col_type(find=['float'])
numeric_cols = base.col_type(find=['numeric'])
date_cols = base.col_type(find=['date'])

# Convert Columns

## Da portare

In [ ]:
# DONE gas_amount, howmuch_pay, total_amount, light_amount to float
# DONE gas_average_col, gas_offer, average_unit_light_cost, average_light_bill_cost, average_unit_gas_cost, average_gas_bill_cost deve essere float non int, basta un astype()
# light_offer è segnato come string perché dovrebbe essere un nome ma nel dataset è un int (what)
# il caso light_offer risolto: in teoria è str ma in pratica è stato anonimizzato, provo il cast to string

In [ ]:
base['light_offer'] = base['light_offer'].astype(str)

In [ ]:
len(base.col_type('string'))

### Date conversion

In [ ]:
#CONVERSIONE COLONNE SU DASK_CUDF 
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}
cols = {
    0:'day',
    1:'month',
    2:'year'
}
#finding columns with date keyword
date_cols = []
for c in df.columns:
    # print(base[c].dtype)
    if 'date' in c.split('_') and base[c].dtype != 'datetime64[ns]':
        date_cols.append(c)

print(f'columns to convert to date: {date_cols}')
#converting these columns
for c in date_cols:

    print(c , df[c].dtype)
    
    a = base.split(c, 2)
    a = a.rename(columns=cols)
    a['month'] = a['month'].str.lower().map(mesi)
    base.get_df()[c] = dd.to_datetime(a, format="%d/%m/%Y")
    # print(a.head())
    
    #df[c] = base.split(c, 2)
    
    # print(base[c].head())

In [ ]:
base.df

### Removing measure unit from strings and converting them into numbers

In [ ]:
# this line can be used to retrieve alle the columns with a number pattern inside them but is too resources consuming
# Due to the small number of columns We preferred to analize them one by one

# base['howmuch_pay'].str.contains('[0-9]+[.,][0-9]*', regex=True).count().compute()
# [0-9]+[.,][0-9]*

In [ ]:
#finding string columns with number inside
to_numeric_cols = []
for c in df.columns:
    if ('average'  in c.split('_') or 'amount' in c.split('_')) and (base[c].dtype in ['object', 'str']):
        # print(base[c].dtype)
        to_numeric_cols.append(c)

# Added manually after column analysis 
to_numeric_cols.append('howmuch_pay')
to_numeric_cols.append('tv')
to_numeric_cols.append('extra_fees')
print(to_numeric_cols)

In [ ]:
# Exctracting measure unit from the column
# Commented bc is time consuming
# base.split('average_gas_bill_cost', 1)[1].value_counts().index.compute()[0]

In [ ]:
# Converting columns from string tu numeric
partitions = base.npartitions()
for c in to_numeric_cols:
    base[c] = base[c].str.extract(r'([0-9]+[,.][0-9]*)', flags=0, expand=False)
    # print(f'{c} measure: {measure}')
    base.replace([c], ',', '.', regex=True)
    base[c] = dd.to_numeric(base[c], errors='coerce', meta=('float'))
# base.get_df().head()['average_gas_bill_cost']

In [ ]:
base.df

In [ ]:
# Strip all the columns in order to remove all the whitespaces 
base.strip(string_cols)

In [ ]:
base.df = base.df.persist()

In [ ]:
base['howmuch_pay'].head()

### Save the modified dataset

In [ ]:
# Save the converted dataset
base.to_parquet('data/mod_parquet')

In [ ]:
base.load_dataset('parquet', path='data/mod_parquet')
base.get_columns_types()

## Da non portare

In [ ]:
# Dovrebbe accorgersi di colonne con similarità > soglia
# TODO --> FUNZIONE DA RIFARE PERCHé USA I SET E QUINDI NON CONFRONTA RIGA-RIGA 
#          MA SET(ELEMENTI_COL1) CON SET(eLEMENTI_COL2)
def get_duplicate_col(df, soglia=0.9):
        """
        Return a list of duplicate columns, if exists.
        Duplicate columns are those which have same values for each row.
        """  
        cols = df.columns.values
        #return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if self.df[cols[i]].equals(self.df[cols[j]])]
        
        return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if len(set(cols[i]) & set(cols[j])) / float(len(set(cols[i]) | set(cols[j]))) > soglia ]

# 1. INGESTION & DISCOVERY

## Analysis DA PORTARE

In [32]:
base['supply_type'].value_counts().compute()

gas           5686288
luce          3684510
gas e luce    1126345
Name: supply_type, dtype: int64

All the 'supply_type' == 'gas' have F1,F2,F3 == 0 as expected
also they all have light_consumption == NaN, we set them to 0

In [33]:
(base[base['supply_type'] == 'gas'][['F1_kWh', 'F2_kWh', 'F3_kWh', 'light_consumption']] == 0).sum().compute()

F1_kWh               5686288
F2_kWh               5686288
F3_kWh               5686288
light_consumption    5686288
dtype: int64

In [34]:
base[base['supply_type'] == 'gas'][['F1_kWh', 'F2_kWh', 'F3_kWh', 'light_consumption']].isna().sum().compute()

F1_kWh               0
F2_kWh               0
F3_kWh               0
light_consumption    0
dtype: int64

In [35]:
base.df['light_consumption'] = base.df['light_consumption'].mask(base['supply_type'] == 'gas', 0)

In [36]:
base.df['light_consumption'].head()

0      0.0
1    219.0
2      0.0
3      0.0
4      0.0
Name: light_consumption, dtype: Float64

### Columns gas related

gas_amount, gas_average_cost, gas_start_date, gas_end_date, gas_consumption, gas_offer, average_unit_gas_cost, average_gas_bill_cost, gas_system_charges, gas_material_cost, gas_transport_cost

In [ ]:
base[['gas_amount', 'gas_average_cost', 'gas_start_date', 'gas_end_date', 'gas_consumption', 'gas_offer', 'average_unit_gas_cost', 'average_gas_bill_cost', 'gas_system_charges', 'gas_material_cost', 'gas_transport_cost']].tail()
# 'gas_average_cost' and 'average_unit_gas_cost' are equivalent

### Columns light related

F1_kWh, F2_kWh, F3_kWh, light_start_date, light_end_date, light_average_cost, light_consumption, light_offer_type, light_offer, light_amount, average_unit_light_cost, average_light_bill_cost, light_system_charges, light_transport_cost, light_material_cost

Riflessioni: 
light_consumption * average_light_bill_cost = light_amount, vero ma c'è un piccolissimo scarto che non li fa combaciare.

light_consumption * average_unit_light_cost = light_material_cost, soldi spesi dall'azienda per comprare l'energia anche qua c'è un piccolissimo scarto

light_amount - light_material_cost = guadagno dell'azienda, a cui bisogna togliere i costi di trasporto, light_system_charges

In [ ]:
base[['F1_kWh', 'F2_kWh', 'F3_kWh', 'light_start_date', 'light_end_date', 'light_average_cost', 'light_consumption', 'light_offer_type', 'light_offer', 'light_amount', 'average_unit_light_cost', 'average_light_bill_cost', 'light_system_charges', 'light_transport_cost', 'light_material_cost']].sample(frac=0.001).compute()

In [15]:
light_related = ['F1_kWh', 'F2_kWh', 'F3_kWh', 'light_start_date', 'light_end_date', 'light_average_cost', 'light_consumption', 'light_offer_type', 'light_offer', 'light_amount', 'average_unit_light_cost', 'average_light_bill_cost', 'light_system_charges', 'light_transport_cost', 'light_material_cost', 'bill_type', 'supply_type']

F1_kWh + F2_kWh + F3_kWh = light_consumption? 

In [37]:
base[(base['light_consumption'] == (base['F1_kWh'] + base['F2_kWh'] + base['F3_kWh']))].shape[0].compute()

10339217

In [116]:
157229 # don't match
10338688 # match

10338688

all the 529 light consumption remaining nan have f1,f2,f3 == 0 so we can set them to 0

In [9]:
base[base['light_consumption'].isna()].shape[0].compute()

5686817

In [10]:
(base[base['light_consumption'].isna()][['F1_kWh', 'F2_kWh', 'F3_kWh']] == 0).sum().compute()

F1_kWh    5686817
F2_kWh    5686817
F3_kWh    5686817
dtype: int64

In [11]:
base['light_consumption'] = base['light_consumption'].fillna(0)

Now let's see how many record doesn't match

In [12]:
base[(base['light_consumption'] == (base['F1_kWh'] + base['F2_kWh'] + base['F3_kWh']))].shape[0].compute()

10339217

In [13]:
base[(base['light_consumption'] != (base['F1_kWh'] + base['F2_kWh'] + base['F3_kWh']))].shape[0].compute()

157229

In [16]:
base[(base['light_consumption'] != (base['F1_kWh'] + base['F2_kWh'] + base['F3_kWh']))][light_related + ['city']].head()

,F1_kWh,F2_kWh,F3_kWh,light_start_date,light_end_date,light_average_cost,light_consumption,light_offer_type,light_offer,light_amount,average_unit_light_cost,average_light_bill_cost,light_system_charges,light_transport_cost,light_material_cost,bill_type,supply_type,city
31,165.0,168.0,187.0,2020-06-01,2020-11-30,<NA>,298.0,light single zone,2301382657258644717,18.59,NaN,NaN,12.47,16.31,-45.68,False,gas e luce,LOZZO DI CADORE
34,108.0,97.0,107.0,2020-06-01,2020-11-30,48.0,199.0,light single zone,13262346701792425167,54.21,0.13,0.27,8.32,15.48,25.48,False,gas e luce,FOSSALTA DI PIAVE
35,153.0,141.0,217.0,2020-02-10,2020-11-26,55.0,49.0,light single zone,5618533779135348807,5.37,0.06,0.11,2.05,-1.31,3.03,False,gas e luce,SANT'OMERO
38,125.0,128.0,135.0,2020-02-01,2020-03-31,<NA>,0.0,light single zone,1919282689015738106,17.96,NaN,NaN,0.0,0.0,-16.32,False,gas e luce,SAN GIOVANNI DI GERACE
44,1356.0,1053.0,1186.0,2019-08-01,2020-11-30,<NA>,-10.0,light single zone,5618533779135348807,2.41,NaN,NaN,-0.76,-0.07,-0.91,False,gas e luce,LIBRIZZI


In [41]:
base[(base['light_consumption'] != (base['F1_kWh'] + base['F2_kWh'] + base['F3_kWh']))]['supply_type'].value_counts().compute()

luce          112085
gas e luce     45144
Name: supply_type, dtype: int64

In [21]:
F_kWh = base[['F1_kWh', 'F2_kWh', 'F3_kWh']]
F_kWh = F_kWh.persist()

In [22]:
# row with all the values not null
F_kWh[~F_kWh.isna().any(axis=1)].shape[0].compute()

10496446

In [25]:
# rows with at least 1 value null
F_kWh[F_kWh.isna().any(axis=1)].shape[0].compute()

697

In [23]:
# row with all values null
F_kWh[F_kWh.isna().all(axis=1)].shape[0].compute()

85

In [4]:
all_na = base[['F1_kWh', 'F2_kWh', 'F3_kWh']].isna().all(axis=1)

85 su 10M di righe hanno sia F1 che F2 che F3 NaN ma il loro light consumption non lo è quindi sostituiamo i valori NaN con un valore medio del tot (F1,F2,F3 = light_consumption/3)
Non l'abbiamo implementato perché è molto difficile cambiare i valori in dask, magari guardiamoci più avanti

In [ ]:
def mean_sub(df):
    all_na = (df['F1_kWh'].isna()) & (df['F2_kWh'].isna()) & (df['F3_kWh'].isna())
    #print(all_na.shape[0])
    df[all_na]['F1_kWh'] = df[all_na]['light_consumption'] / 3
    return df

base.df[all_na].map_partitions(mean_sub, meta=base.df).compute()

In [26]:
F_kWh['sum'] = F_kWh.sum(axis=1)

In [ ]:
F_kWh[F_kWh[['F1_kWh', 'F2_kWh', 'F3_kWh']].isna().all(axis=1)].compute()

In [59]:
base.df = base.df.assign(new_light_consumption = base[['F1_kWh', 'F2_kWh', 'F3_kWh']].sum(axis=1))

In [73]:
base.df[(base['new_light_consumption'] != base['light_consumption'])].tail()[['F1_kWh', 'F2_kWh', 'F3_kWh', 'light_consumption', 'new_light_consumption']]

,F1_kWh,F2_kWh,F3_kWh,light_consumption,new_light_consumption
538637,372.0,351.0,353.0,564.0,1076.0
538682,341.0,356.0,208.0,649.0,905.0
538698,276.0,391.0,563.0,650.0,1230.0
538703,163.0,261.0,671.0,254.0,1095.0
538711,509.0,397.0,547.0,784.0,1453.0


If the F1 + F2 + F3 is < 0 is light_average_cost, average_unit_light_cost, average_light_bill_cost always NaN ? 

In [ ]:
157778

In [77]:
3684510+1126345

4810855

### All the record with city == VIETRI DI POTENZA have address and nominative null and all the addresses and nominative null values are the VIETRI DI POTENZA records

In [ ]:
#esempio
base.locate_null_values('address').shape[0].compute()

In [ ]:
base[base['city'] == 'VIETRI DI POTENZA'].shape[0].compute()

In [ ]:
base[base['nominative'].isna()].shape[0].compute()

## Locate Missing Values DA PORTARE

In [ ]:
# LOCATE MISSING VALUES
#rows = df.shape[0]
#wait(ddf.shape[0])
#print("Total rows", rows)

# These are the colums with at least one missing value
column_count_nan = {}
# compute the presence of null value on time instead every iteration for performance reasons
na = base.get_df().isna()
val = na.any().compute()

res = na.sum().compute()
column_count_nan = json.loads(res.to_json())
column_count_nan

In [ ]:
import json

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
    
with open('data/initial_null_val.json', 'w+') as f:
    # this would place the entire output on one line
    # use json.dump(lista_items, f, indent=4) to "pretty-print" with four spaces per indent
    json.dump(column_count_nan, f, cls=NpEncoder, indent=2)

In [ ]:
import json 

with open('data/initial_null_val.json', "r") as read_file:
    column_count_nan = json.load(read_file)

In [ ]:
column_with_nan = []

perc = 80
soglia = ((perc * base.get_df().shape[0]) / 100).compute()
print(f'soglia {soglia}')

for c, val in column_count_nan.items():
    #print(f'{c}')
    if val > soglia:
        print(f'{c} val: {val}')
        column_with_nan.append(c)
        
column_with_nan

# Si può notare come nessuna colonna abbia il 100% di valori nulli, ma ci sono altresì tante 
# colonne con un numero elevato di nan. Poche colonne hanno un numero basso di nan.

### Analisi extra (utile estenderla all'intero dataset)

In [ ]:
#Visto che address e nominative hanno lo stesso numero di NA, vale la 
#pena controllare ed eventualmente eliminare le righe?

base.locate_null_values('nominative').shape[0].compute()

In [ ]:
base.locate_null_values('address').shape[0].compute()

In [ ]:
df[df['city'] == 'VIETRI DI POTENZA'].shape[0].compute()

Not all the user code are repeated, but even in the repeated we can't be sure if the 2 records correspond, this because the real name are hased and possible collission can exists

In [ ]:
df['user_code'].value_counts().shape[0].compute()

In [ ]:
# PROVO A VEDERE SE SONO PRESENTI ALTRI RECORDS DEGLI UTENTI A CUI MANCANO NOMINATIVE E ADDRESS
v = df['user_code'].value_counts()
v[v.gt(1)].index.compute()

In [ ]:
# DATASET CON GLI user_code CHE COMPAIONO PIù DI UNA VOLTA
df[df['user_code'].isin(v[v.gt(1)].index.compute())].compute()

In [ ]:
# Le date potrebbero fornirci informazioni sulla billing_frequency, o viceversa, come lo stabiliamo?
# Cosa indica la colonna dell'emission_date??

#NOTA PER DASK, SE FACCIAMO 0 CI TORNA IL PRIMO RECORD DI TUTTE LE 52 PARTIZIONI
df.loc[0, 'billing_frequency'].compute()

## Locate Outliers DA PORTARE 

In [ ]:
base['F1_kWh'].quantile(0.1).compute()

In [ ]:
#CERTE COLONNE CON IL TIPO DI DATO PARTICOLARE TIPO FLOATDTYPE64 VANNO CONVERTITE IN TIPI COMPRENSIBILI
base.locate_outliers('F1_kWh').head()

In [ ]:
type = df.dtypes
num = type[type!='object'][type!='bool']
num = num.index.to_list()
num

In [ ]:
tmp = ddf.locate_outliers("F1_kWh")
tmp = tmp.compute()
tmp.shape

In [ ]:
for col in num:
    tmp = base.locate_outliers(col)
    tmp = tmp.compute()
    
    print(f"{col} has {tmp.shape[0]} outliers")

## Sort Data DA PORTARE

In [ ]:
base.sort("F1_kWh").head()

### Bazze mirco

In [ ]:
tmp = ddf.sort_index("F1_kWh").compute()
# not working

In [ ]:
idx = tmp.index.to_arrow().to_pylist()
# too mutch long list

In [ ]:
tmp.merge(ddf.df, on=["F1_kWh"], how="left")

# 2. VALIDATION

In [ ]:
''' 
+++++ DATA VALIDATION +++++
    • Check data range
    • Check column uniqueness
    • Find data-mismatched data types
'''

## Check Data Range DA PORTARE TUTTE (mi tirava fare un sottogruppo)

### Age nulla per PIVA

In [108]:
# Controllo se chi ha P ha età nulla
import pandas as pd
df['age'] = df['age'].mask((df['sex']=='P') & (df['age']>=0), np.nan)
df[(df['sex']=='P')][['sex','age']].sample(frac=0.1).compute()

,sex,age
119293,P,NaN
20190,P,NaN
62654,P,NaN
21391,P,NaN
40094,P,NaN
...,...,...
524905,P,NaN
527378,P,NaN
531331,P,NaN
533277,P,NaN


### Age >= 18

In [ ]:
# Age >= 18
# Age non ha valori nulli (già controllato in precedenza IN TEORIA)
# Dovrebbe corrispondere al numero di PIVA
df['age'].isna().sum().compute()

In [ ]:
# Che fare con i minorenni? Li mettiamo tutti a 18 anni?
df[(df['age'] < 18)].shape[0].compute()

### Translation in English

In [ ]:
df['supply_type'].value_counts(dropna=False).compute()

In [ ]:
# Dobbiamo metterli in inglese? Si può fare con un map
ing = {
    'luce': 'light',
    'gas e luce': 'gas and light',
    'gas' : 'gas',
}
df['supply_type'] = df['supply_type'].map(ing)

In [ ]:
df['supply_type'].value_counts(dropna=False).compute()

### Data control

### billing_freq & bill_type Analysis

In [ ]:
# Che fare qua? Billing frequency (‘monthly’, ‘quarterly’, etc.)
# Si potrebbero calcolare
df['billing_frequency'].value_counts(dropna=False).compute()

In [ ]:
# Kind of invoice (False means a “standard bill”)
df['bill_type'].value_counts(dropna=False).compute()

### light_offer_type Analysis

In [ ]:
# Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
df['light_offer_type'].value_counts(dropna=False).compute()

In [ ]:
# I 2 valori di light mi puzzano, famo uno strip?
df['light_offer_type'] = df['light_offer_type'].str.strip()
df['light_offer_type'].value_counts(dropna=False).compute()
# GHAVEVO RAGIONE Vè

## Check Column Uniqueness DA PORTARE E DA VERIFICARE

In [ ]:
# The story changes here, we got these pairs as candidates as duplicate columns 
# (those which have same values for each row.)

# ATTENZIONE!! Probabile che le date del gas siano invertite, trovare un modo per confermarlo

In [ ]:
## Sserve il date formatter!!
df['emission_date'].head()

In [ ]:
# VERY time consuming, expecially with DASK, with DASK_CUDF It is a lot faster but It takes a lot of time as well
# base.get_duplicate_columns()
# Ne vengono individuate solo 5 di coppie 100% fedeli, 
# ma cambia la questione se abbassiamo la soglia? Ci sono degli errori sopra perchè 
# quelle 100% uguali non vengono riconosciute, mentre qua sì
duplicates = [('date', 'light_start_date'),('date', 'gas_end_date'),('light_start_date', 'gas_end_date'),('light_end_date', 'gas_start_date'),('gas_average_cost', 'average_unit_gas_cost')]
duplicates

## Find Data Mismatch

In [ ]:
df['date'] == df['light_start_date']

In [68]:
df[['light_start_date', 'light_end_date']][base['light_start_date'] <= base['light_end_date']].shape[0].compute()
# i record mancanti sono delle righe con le date null, ma di base vanno bene

10489272

In [66]:

10497143 

10497143

In [72]:
df[['gas_start_date', 'gas_end_date']][base['gas_start_date'] < base['gas_end_date']].shape[0].compute()
# da invertire

0

In [73]:
base.rename_columns({
    'gas_start_date':'gas_end_date',
    'gas_end_date':'gas_start_date'
})

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,date,light_start_date,light_end_date,tv,gas_amount,gas_average_cost,light_average_cost,emission_date,supply_type,gas_end_date,gas_start_date,extra_fees,gas_consumption,light_consumption,gas_offer,light_offer_type,light_offer,howmuch_pay,total_amount,light_amount,average_unit_light_cost,average_light_bill_cost,average_unit_gas_cost,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
npartitions=57,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,Float64,Float64,Float64,object,object,object,object,int64,object,datetime64[ns],datetime64[ns],datetime64[ns],float64,float64,float64,Float64,datetime64[ns],object,datetime64[ns],datetime64[ns],float64,Float64,Float64,uint64,object,object,float64,float64,float64,float64,float64,float64,float64,object,object,object,Float64,Float64,Float64,Float64,Float64,Float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
base['tv'].head()

In [42]:
# Substitute False with standard bill

base['bill_type'] = base['bill_type'].replace('False', 'standard bill')

In [43]:
base['bill_type'].value_counts().compute()

standard bill             10446508
variation note               30477
change note correction       19616
cancellation bill              542
Name: bill_type, dtype: int64

In [45]:
# Non c'è nessuna riga tutta nulla
df.dropna(how='all')

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,CIPRESSA,C23B8FC235DF5479FB28D81E827A59B819BB0748,0298E5E66B5653CDAACADD156261A2A916A56C7E,F,81,63D286C61D20D76E9C9317BEB8644D5EE45134E4,...,NaN,5AE3987CBA311BCAADC6274D28A7FB14B13B53C3,None,False,-0.06,<NA>,0.06,<NA>,-0.2,<NA>
1,1,81.0,62.0,76.0,ROCCAVIVARA,CBE6B021F41D589269FBC724C131CBFC9905D950,FDAE2B1E0934BB76255A727CC9F3F00AA39982D9,M,38,8053C7982DE8A18746A8F9F0D1DC4B0C33CAC0FA,...,NaN,5F88D510670063B20E789E6453BC2F3FB0BB67F7,None,False,<NA>,9.15,<NA>,5.68,<NA>,23.13
2,2,0.0,0.0,0.0,VIDRACCO,BFAEB566B3BB78B9ABD4F86DB3A78BFD8489013B,FEBE7F29FB854C477AFA7073C70F0E6EA81C6B00,M,53,4F053D54B9F1B6FE565498E7FA726501FD8FDD18,...,NaN,D4B8928760E729127EB4EB532C86127325FBA468,None,False,1.24,<NA>,-10.13,<NA>,-7.13,<NA>
3,3,0.0,0.0,0.0,MEZZOLOMBARDO,CBD30C382C72ED4A3D9DC9D11C13C155930E7C66,9220A3E83BDCE622F70A1456C7898687FD3EDAC8,M,50,115C7D34A79C66AFCDB4EB65262595B197B0F861,...,1.08,BD4DEF66D7EF2D3D683D9C381D2359AD24B41D9C,None,False,-0.88,<NA>,23.68,<NA>,17.66,<NA>
4,4,0.0,0.0,0.0,PINETO,4D17C711CADED6C1E9BD459088B4B80F24575FE7,642C21354EF6017D13220517E2A25D4A0442D4C5,M,65,C7734B97A212004CEA25956D8FDFEF068CD2B8F6,...,0.86,C0622AFD22384A4B54CD1ACAF7D988B89853A5AC,None,False,14.71,<NA>,141.57,<NA>,63.59,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542441,2512812,0.0,0.0,0.0,CASALZUIGNO,FE1047ECA6069845A48B36ABAD2A725C545A3ACD,59801133A00E8156E2FCB182350344C222C625E9,M,66,CCC653F93977E0948C0E71592DC0B35D8088BF80,...,1.23,B90850838689F4530C6B2403ABF9D6A7DB4A6C18,bimester,False,-2.09,<NA>,41.72,<NA>,34.95,<NA>
542442,2512813,0.0,0.0,0.0,ACQUANEGRA CREMONESE,31432A91213A1877995BE91793761AB04AB2C1D9,ED3368875442BC333089F2EF36CEF71865954BCC,F,46,CEF01180CFC826BD1B1D940A77532A09A4E13761,...,1.29,6CCC81E05B7ED7401945CB8FB42126B242AB443C,bimester,False,-1.1,<NA>,41.64,<NA>,34.03,<NA>
542443,2512814,0.0,0.0,0.0,SAN LUPO,B3260B1FAEE19CB253D0B27B10CDCE0F40606AA4,959F1C15E6FD24DF29439A2B4683EC280BB95466,M,45,255D0B199EAAC6E72B483D38E4EA85C643109633,...,1.58,5DC1024A436D07BDF4C4853A752CF89775C3F295,bimester,False,-3.83,<NA>,30.81,<NA>,21.25,<NA>
542444,2512815,0.0,0.0,0.0,VERNATE,9AADECE289CFA1A60FAE6F30354B16DEEE1644A0,D3FD778C3F95AE2DAB5B9A1FE11FC14728D85164,M,64,A6E6C0A1A17CE15A4A822F516E5F6B1E8CD68B23,...,1.09,454279C1315FE7D78684138FB9998240B99D86DC,bimester,False,-0.22,<NA>,67.14,<NA>,27.61,<NA>


# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #
# BOOKMARK #

# 3. STRUCTURING

In [ ]:
''' 
+++++ DATA STRUCTURING +++++
    • Change column data types
    • Delete, split or merge columns
    • Pivot and unpivot
'''

## Change Column Data Types 

In [ ]:
# aggregabile con lo step precedente
# L'HO FATTO SOPRA

## Delete, Split or Merge Columns

In [ ]:
# rimozione colonne duplicate 

## Pivot and Unpivot

In [ ]:
# non saprei cosa farci

# 4. ENRICHMENT

In [ ]:
''' 
+++++ DATA ENRICHMENT +++++
    • Calculate columns using expressions
    • Set primary key column
    • Join or append DataFrames
    • Group by and aggregate records
    • Scale column values into a certain range
    • Encode categorical data (one-hot encoding and label encoding)
'''

## Calculate Columns Using Expressions

## Set PK column

# 5. FILTERING

In [ ]:
''' 
+++++ DATA FILTERING +++++
    • Sample rows
    • Select a subset of rows through a user-defined query
'''

## Sample Rows

In [ ]:
df.sample(20)

In [ ]:
# SELECT A SUBSET OF ROWS?!?

# 6. CLEANING

In [ ]:
''' 
+++++ DATA CLEANING +++++
    • Change data format and case
    • Deduplicate data
    • Missing value imputation
    • Find and replace values
'''

In [ ]:
# CHANGE DATA FORMAT AND CASE
# data format dovrebbe già essere fatto a questo punto
# case pure se vogliamo aver fatto tutto per bene

In [ ]:
# DEDUPLICATE DATA
# controllo se ho più bollette uguali dello stesso cliente

In [ ]:
ddf

In [ ]:
# MISSING VALUE IMPUTATION
# qua? Spero di averne pochissimi nel caso

In [ ]:
# FIND AND REPLACE VALUES
# ossia?

In [ ]:
x = np.nan
float(x)